In [ ]:
#learn more about pyterrier: https://pyterrier.readthedocs.io/en/latest/

Aufgabe: Bitte nutzt jetzt die Json Datei, die ihr durch Aufgabe 1 erhalten habt. Führt den kompletten Indexierungsvorgang mit PyTerrrier durch. Der Python Code der von euch gefordert wird, ist durch ??? markiert.
Stellt euch die folgenden Fragen:
* Welche Spalten aus dem Dataframe könnten als Id welche als Text Feld benutzt?
* Welche Felder sollen indexiert werden?
* Was ist wichtig für die Indexierungspipeline?
* Zusatzaufgabe: Wie könnten beim Suchvorgang Publikationen gefilter werden, deren Erscheinungsjahr nicht zwischen 2015 und 2020 ist? Passt dazu eure Definition der Suchmaschine an.


In [1]:
import pyterrier as pt
import pandas as pd
import pickle
import numpy as np

In [2]:
path = "literature.json"
data = pd.read_json(path)

In [3]:
data

,query,title,url,year,authors,abstract,source
0,artificial intelligence,Artificial Intelligence in Agriculture,https://www.ijtsrd.com/engineering/electrical-...,2021,"[Matthew N. O. Sadiku, Sarhan M. Musa, Abayomi...",Artificial Intelligence is one of the emerging...,bibsonomy
1,artificial intelligence,Artificial Intelligence Benefit and Risks,https://www.ijtsrd.com/computer-science/artifi...,2020,[Seeta M. Chauhan],this article demonstrate Disadvantage of artif...,bibsonomy
2,artificial intelligence,Artificial Intelligence in Power Station,https://www.ijtsrd.com/engineering/electrical-...,2019,"[P. Naveen, S. Nikitha, P. Sudeesh, V. Vaishnavi]",With increased competitiveness in power genera...,bibsonomy
3,artificial intelligence,Artificial Intelligence Based Training and Pla...,http://www.ijtsrd.com/computer-science/artific...,2018,"[Krishanu Deb, Pankaj Agrawal, Harish Nawale, ...",Training and placement cell in colleges is to ...,bibsonomy
4,artificial intelligence,The Significance of Artificial Intelligence in...,https://www.ijtsrd.com/computer-science/artifi...,2023,[Dr. Atul Kumar Mishra],"In an increasingly digitalized world, the util...",bibsonomy
...,...,...,...,...,...,...,...
4737,large language model,Phonologically Aware Neural Model for Named En...,https://www.semanticscholar.org/paper/f1a8ff55...,2016,"[Akash Bharadwaj, David R. Mortensen, Chris Dy...",Named Entity Recognition is a well established...,semantic_scholar
4738,large language model,Large-Scale Distributed Language Modeling,https://www.semanticscholar.org/paper/591080c3...,2007,"[Ahmad Emami, K. Papineni, Jeffrey Scott Soren...",A novel distributed language model that has no...,semantic_scholar
4739,large language model,BERT for Joint Intent Classification and Slot ...,https://www.semanticscholar.org/paper/476029ac...,2019,"[Qian Chen, Zhu Zhuo, Wen Wang]",Intent classification and slot filling are two...,semantic_scholar
4740,large language model,Language and Translation Model Adaptation usin...,https://www.semanticscholar.org/paper/b281a9d0...,2008,"[M. Snover, B. Dorr, R. Schwartz]","Traditionally, statistical machine translation...",semantic_scholar


In [4]:
#initialise pyterrier
if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
#pyterrier exspects a docno field
data['docno'] = data["url"]
data['text'] = data["abstract"]

#year field should be string to simplify indexing 
data['year'] = data['year'].astype('str')

In [6]:
#transfrom dataframe into list of dictionaries
data_dict = data.to_dict(orient='records')

In [8]:
index_folder_mult = "./ai_llm_index"

fields = ["title","abstract"]

indexer_mult = pt.IterDictIndexer(index_folder_mult, meta={'docno': 1024, 'text': 4096, 'year' : 1024}, overwrite=True)
index_ref_mult = indexer_mult.index(data_dict, fields=fields)

print(f"path to our index: {index_ref_mult.toString()}")

path to our index: ./ai_llm_index/data.properties


In [9]:
index_mult = pt.IndexFactory.of(index_ref_mult)

In [10]:
print(index_mult.getCollectionStatistics().toString())

Number of documents: 4742
Number of terms: 11413
Number of postings: 175235
Number of fields: 2
Number of tokens: 258580
Field names: [title, abstract]
Positions:   false



In [11]:
search_engine_mult = pt.BatchRetrieve(index_mult, wmodel="TF_IDF")

In [12]:
res = search_engine_mult.search("sentiment analysis")

In [13]:
#save our data
pickle.dump(data, open("data_exercise.pkl", "wb"))

In [14]:
search_engine = pt.BatchRetrieve(index_mult, wmodel="TF_IDF", metadata=["docno", "year"]) 

rank_filter = pt.apply.generic(lambda res : res[res["rank"] < 10])

search_engine_year_filter = search_engine >> rank_filter

In [19]:
search_engine_mult.search("sentiment analysis")

,qid,docid,docno,rank,score,query
0,1,1219,http://dblp.uni-trier.de/db/conf/ai/ai2011.htm...,0,9.094175,sentiment analysis
1,1,1694,https://www.ijtsrd.comengineering/computer-eng...,1,8.862863,sentiment analysis
2,1,744,https://www.ijtsrd.comcomputer-science/artific...,2,8.828624,sentiment analysis
3,1,1243,http://dblp.uni-trier.de/db/conf/ai/ai2018.htm...,3,8.679697,sentiment analysis
4,1,2153,,4,8.679697,sentiment analysis
...,...,...,...,...,...,...
382,1,3514,https://www.semanticscholar.org/paper/3c151694...,382,0.929448,sentiment analysis
383,1,2009,http://dx.plos.org/10.1371/journal.pone.020234...,383,0.920177,sentiment analysis
384,1,4054,https://www.semanticscholar.org/paper/b241da5a...,384,0.882052,sentiment analysis
385,1,4094,https://www.semanticscholar.org/paper/0c42c433...,385,0.497975,sentiment analysis


In [20]:
search_engine.search("sentiment analysis")

,qid,docid,docno,year,rank,score,query
0,1,1219,http://dblp.uni-trier.de/db/conf/ai/ai2011.htm...,2011,0,9.094175,sentiment analysis
1,1,1694,https://www.ijtsrd.comengineering/computer-eng...,2021,1,8.862863,sentiment analysis
2,1,744,https://www.ijtsrd.comcomputer-science/artific...,2021,2,8.828624,sentiment analysis
3,1,1243,http://dblp.uni-trier.de/db/conf/ai/ai2018.htm...,2018,3,8.679697,sentiment analysis
4,1,2153,,2023,4,8.679697,sentiment analysis
...,...,...,...,...,...,...,...
382,1,3514,https://www.semanticscholar.org/paper/3c151694...,2019,382,0.929448,sentiment analysis
383,1,2009,http://dx.plos.org/10.1371/journal.pone.020234...,2018,383,0.920177,sentiment analysis
384,1,4054,https://www.semanticscholar.org/paper/b241da5a...,2020,384,0.882052,sentiment analysis
385,1,4094,https://www.semanticscholar.org/paper/0c42c433...,2016,385,0.497975,sentiment analysis


In [49]:
search_engine = pt.BatchRetrieve(index_mult, wmodel="TF_IDF", metadata=["docno", "year"]) 
rank_filter = pt.apply.generic(lambda res : res[res["rank"] < 10])

rank_filter2 = pt.apply.generic(lambda res : res[res["year"] <= '2015'])
rank_filter3 = pt.apply.generic(lambda res : res[res["year"] >= '2020'])

search_engine_year_filter = search_engine >> rank_filter >> rank_filter2 >> rank_filter3

In [50]:
search_engine_year_filter.search("sentiment analysis")

,qid,docid,docno,year,rank,score,query
